# HW 1

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [4]:
import itertools
from astropy import units
import sympy
import pandas
from scipy import integrate
import numpy
import plotly.express as px

from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature, geodesic
from pystein.utilities import tensor_pow as tpow, full_simplify, boundary_filter

ModuleNotFoundError: No module named 'pystein'

In [ ]:
show_plots = True

## Exercises

### B1 - Polar Coords for $\mathbb{R}^2$

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

##### Cartesian

In [ ]:
x, y = sympy.symbols('x, y', nonnegative=False)
cs = coords.CoordSystem('cartesian', P, [x, y])
dx, dy = cs.base_oneforms()
ds2 = tpow(dx, 2) + tpow(dy, 2)
g_cart = metric.Metric(twoform=ds2)
g_cart

##### Polar

In [ ]:
r, theta = sympy.symbols('r theta', nonnegative=True)
cs = coords.CoordSystem('polar', P, [r, theta])
dr, dtheta = cs.base_oneforms()
ds2 = tpow(dr, 2) + r ** 2 * tpow(dtheta, 2)
g1 = metric.Metric(twoform=ds2)
g1

#### Compute curvature components

In [ ]:
crs, rmns, rcs = curvature.compute_components(g1)

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rmns)

In [ ]:
curvature.display_components(rcs)

#### Parallel Transport

In [ ]:
# Unit circle parameterized such that x(0) == x(1)
param = sympy.symbols('lambda')
r_0 = sympy.symbols('r_0', real=True)
curve = [
    r_0,
    2 * sympy.pi * param,
]

In [ ]:
soln = geodesic.parallel_transport_soln(param, curve, g1)

In [ ]:
soln.soln[0]

In [ ]:
g1.angle(soln.vec(0), soln.vec(1))

#### Compute Geodesics

In [ ]:
full_simplify(geodesic.geodesic_equation(0, sympy.symbols('lambda'), g1))

In [ ]:
full_simplify(geodesic.geodesic_equation(1, sympy.symbols('lambda'), g1))

##### Visualize Cartesian

In [ ]:
ls = numpy.arange(0, 2, 0.01)
init = (0.0, 0.0, 0.1, 0.1) 
df1cart = geodesic.numerical_geodesic(g_cart, init, ls)

In [ ]:
ls = numpy.arange(0, 2, 0.001)
df1scart = geodesic.numerical_sampler(g_cart, ls, (0, 0), num_angles=6)

In [ ]:
if show_plots:
    fig = px.scatter(df1scart, x="x", y='y', 
                     color='theta_0',
                     title="B1 Cart Geodesic",
                     height=600, width=600)

    fig.show()

##### Visualize Polar

In [ ]:
ls = numpy.arange(0, 2, 0.001)
r_0 = 1.0 
dfs = [geodesic.numerical_geodesic(g1, (r_0, theta_0, 0.0, numpy.pi/2), ls).assign(theta_0=theta_0) 
       for theta_0 in numpy.arange(0.0, 2 * numpy.pi, 0.2)]
df = pandas.concat(dfs, axis=0)

In [ ]:
df['theta'] = numpy.mod(df['theta'], 2*numpy.pi)
# df['lam'] = ls

In [ ]:
if show_plots:
    fig = px.scatter(df, x="theta", y='r', 
                     color='theta_0',
                     title="Sample Geodesic")

    fig.show()

### B2 2-Sphere

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

theta, phi, a = sympy.symbols('theta phi a', nonnegative=True)
cs = coords.CoordSystem('spherical', P, [theta, phi])
dtheta, dphi = cs.base_oneforms()
ds2 = a**2 * (tpow(dtheta, 2) + sympy.sin(theta)**2 * tpow(dphi, 2))
g2 = metric.Metric(twoform=ds2)
g2

#### Compute Components

In [ ]:
g2.matrix

In [ ]:
g2.inverse.matrix

In [ ]:
crs, rmn, rcs = curvature.compute_components(g2)

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rmn)

In [ ]:
curvature.display_components(rcs)

In [ ]:
full_simplify(curvature.ricci_scalar(metric=g2))

#### Parallel Transport

In [ ]:
param, theta_0 = sympy.symbols('lambda theta_0')

curve = [
    theta_0,
    param,
]

In [ ]:
geodesic.parallel_transport_equation(0, curve, param, g2)

In [ ]:
full_simplify(geodesic.parallel_transport_equation(1, curve, param, g2))

#### Geodesics

##### Geodesic Equations

In [ ]:
full_simplify(geodesic.geodesic_equation(0, sympy.symbols('lambda'), g2))

In [ ]:
full_simplify(geodesic.geodesic_equation(1, sympy.symbols('lambda'), g2))

##### Single Geodesic

In [ ]:
ls = numpy.arange(0, 8, 0.0001)
init = (0.01, 0.001, 3.14/4, 0.0)
st = 2
sp = 2
df2 = geodesic.numerical_geodesic(g2, (numpy.pi/2, 0.0, numpy.pi/st, numpy.pi/sp), ls)
#        for theta_0 in numpy.arange(0.01, 3.14, 0.5)]

In [ ]:
df2['theta'] = numpy.mod(df2['theta'], 2*numpy.pi)
df2['phi'] = numpy.mod(df2['phi'], numpy.pi)
df2 = df2.reset_index().rename(columns={'index': 'order'})
df2['lam'] = ls

In [ ]:
if show_plots:
    fig = px.scatter(df2, x="theta", y='phi', 
                     color='order',
                     title="B2 Geodesic")

    fig.show()

##### Multiple Initial Conditions

In [ ]:
ls = numpy.arange(0, .04, 0.00001)
st = 2
sp = .01
dfs2 = [geodesic.numerical_geodesic(g2, (theta_0, 0.0, numpy.pi/st, numpy.pi/sp), ls).assign(theta_0=theta_0) 
       for theta_0 in list(numpy.arange(numpy.pi/6, 5*numpy.pi/6, 0.1)) + 
       list(numpy.arange(numpy.pi/6 + numpy.pi, 5*numpy.pi/6 + numpy.pi, 0.1))]
#        for theta_0 in numpy.arange(0.01, 3.14, 0.5)]
df2b = pandas.concat(dfs2, axis=0)

In [ ]:
df2b['theta'] = numpy.mod(df2b['theta'], 2*numpy.pi)
df2b = df2b.reset_index().rename(columns={'index': 'order'})

In [ ]:
lim_df2b = boundary_filter(df2b, theta=(0, 2*3.14), phi=(0, 3.14))

In [ ]:
if show_plots:
    fig = px.scatter(lim_df2b, x="theta", y='phi', 
                     color='theta_0',
                     title="B2 Geodesics")

    fig.show()

### B3 2-Sphere Sch

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

rho, phi, a = sympy.symbols('rho phi a', nonnegative=True)
cs = coords.CoordSystem('schw', P, [rho, phi])
drho, dphi = cs.base_oneforms()
ds2 = a**2 * ( (1 / (1 - rho**2)) * tpow(drho, 2) + rho ** 2 * tpow(dphi, 2))
g3 = metric.Metric(twoform=ds2)
g3

#### Geodesics

In [ ]:
full_simplify(geodesic.geodesic_equation(0, sympy.symbols('lambda'), g3))

In [ ]:
full_simplify(geodesic.geodesic_equation(1, sympy.symbols('lambda'), g3))

In [ ]:
# init = (numpy.sin(numpy.pi/4), 0.0, numpy.cos(numpy.pi/4), numpy.pi/4)
init = (numpy.sin(0), 0.0, numpy.cos(numpy.pi/4), numpy.pi/4)
lambdas = numpy.arange(0, 2.1, 0.001)
df3 = geodesic.numerical_geodesic(g3, init, lambdas)

In [ ]:
df3['phi'] = numpy.mod(df3['phi'], numpy.pi)
df3 = df3.reset_index().rename(columns={'index': 'order'})
df3['lam'] = lambdas

In [ ]:
if show_plots:
    fig = px.scatter(df3, x="rho", y='phi', 
    #                  color='init',
                         title="B3 Geodesic")

    fig.show()

### B4 - Embedded Surface - General Case

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

x, y = sympy.symbols('x y', nonnegative=False)
_coords = [x, y]
cs = coords.CoordSystem('Cartesian', P, [x, y])
dx, dy = cs.base_oneforms()
_1forms = [dx, dy]

f = sympy.Function('f')(x, y)

ds2 = []
for i in range(2):
    for j in range(2):
        ds2.append(((1 if i == j else 0) + sympy.diff(f, _coords[i]) * sympy.diff(f, _coords[j])) 
                   * sympy.diffgeom.TensorProduct(_1forms[i],  _1forms[j]))
ds2 = sum(ds2)
g4 = metric.Metric(twoform=ds2)
g4

#### Compute Components

In [ ]:
g4.matrix

In [ ]:
sympy.det(g4.matrix)

In [ ]:
g4.inverse.matrix

In [ ]:
crs, rms, rcs = curvature.compute_components(g4)

In [ ]:
# Simplify notation
subscript_notation = [
    (sympy.diff(sympy.diff(f, x), x), sympy.symbols('f_xx')),
    (sympy.diff(sympy.diff(f, y), y), sympy.symbols('f_yy')),
    (sympy.diff(sympy.diff(f, x), y), sympy.symbols('f_xy')),
    (sympy.diff(f, x), sympy.symbols('f_x')),
    (sympy.diff(f, y), sympy.symbols('f_y')),
]

In [ ]:
crs = [(c[0], c[1].subs(subscript_notation)) for c in crs]
rms = [(c[0], c[1].subs(subscript_notation)) for c in rms]
rcs = [(c[0], c[1].subs(subscript_notation)) for c in rcs]

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rms)

In [ ]:
curvature.display_components(rcs)

Clean Expr

In [ ]:
simplified = full_simplify(curvature.ricci_scalar(g4)).subs(subscript_notation)
simplified

#### Fun Examples

Paraboloid

In [ ]:
a, b = sympy.symbols('a b')
subs = {f: x**2/a + y**2/b}

In [ ]:
full_simplify(full_simplify(curvature.ricci_scalar(g4)).subs(subs))

### B5 - Embedded Surface - Sphere

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0**2 - x**2 - y**2)}
g5 = metric.Metric(twoform=g4.twoform.subs(subs))
g5

#### Curvature

In [ ]:
full_simplify(curvature.ricci_scalar(g5, simplify_intermediate=True))

#### Parallel Transport

In [ ]:
param = sympy.symbols('\\lambda')

curve = [
    sympy.cos(2 * sympy.pi * param),
    sympy.sin(2 * sympy.pi * param),
]

In [ ]:
full_simplify(geodesic.parallel_transport_equation(0, curve, param, g5))

In [ ]:
full_simplify(geodesic.parallel_transport_equation(1, curve, param, g5))

#### Geodesic

##### Geodesic Equations

In [ ]:
full_simplify(geodesic.geodesic_equation(0, sympy.symbols('lambda'), g5))

In [ ]:
full_simplify(geodesic.geodesic_equation(1, sympy.symbols('lambda'), g5))

##### Visual

In [ ]:
a_0_val = 1.0
g5_num = metric.Metric(twoform=g5.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
df5 = geodesic.numerical_sampler(g5_num, ls, (0.5, 0.5), tangent_scale=0.1)

In [ ]:
lim_df5 = boundary_filter(df5, x=(-1, 1), y=(-1, 1))

In [ ]:
if show_plots:
    fig = px.scatter(lim_df5, x="x", y='y', 
                     color='theta_0',
                     title="B5 Geodesic", 
                     height=600, width=600)
    fig.show()

### B6 - Embedded Surface - Hyperboloid

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0**2 - x**2 + y**2)}
g6 = metric.Metric(twoform=g4.twoform.subs(subs))
g6

In [ ]:
g6.matrix.doit()

#### Curvature

In [ ]:
full_simplify(curvature.ricci_scalar(g6, simplify_intermediate=True))

#### Parallel Transport

In [ ]:
full_simplify(geodesic.parallel_transport_equation(0, curve, param, g6))

In [ ]:
full_simplify(geodesic.parallel_transport_equation(0, curve, param, g6))

#### Geodesic

In [ ]:
a_0_val = 1.0
g6_num = metric.Metric(twoform=g6.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
df6 = geodesic.numerical_sampler(g6_num, ls, (0.5, 0.5), tangent_scale=0.1)

In [ ]:
lim_df6 = boundary_filter(df6, x=(-1, 1), y=(-1, 1))

In [ ]:
if show_plots:
    fig = px.scatter(lim_df6, x="x", y='y', 
                     color='theta_0',
                     title="B6 Geodesic",
                     height=600, width=600)

    fig.show()

### B7 - Embedded Surface - Cylinder

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0**2 - y**2)}
g7 = metric.Metric(twoform=g4.twoform.subs(subs))
g7

In [ ]:
g7.matrix.doit()

#### Curvature

In [ ]:
crs, rmn, rcs = curvature.compute_components(g7)

In [ ]:
curvature.display_components(crs)

In [ ]:
full_simplify(curvature.ricci_scalar(g7))

#### Parallel Transport

In [ ]:
full_simplify(geodesic.parallel_transport_equation(0, curve, param, g7))

In [ ]:
full_simplify(geodesic.parallel_transport_equation(1, curve, param, g7))

#### Geodesic

In [ ]:
a_0_val = 1.0
g7_num = metric.Metric(twoform=g7.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
df7 = geodesic.numerical_sampler(g7_num, ls, (0.5, 0.5), tangent_scale=0.1)

In [ ]:
lim_df7 = boundary_filter(df7, x=(-1, 1), y=(-1, 1))

In [ ]:
if show_plots:
    fig = px.scatter(lim_df7, x="x", y='y', 
                     color='theta_0',
                     title="B7 Geodesic",
                     height=600, width=600)
    fig.show()

### B8 - Embedded Surface - Cone

#### Substitution

In [ ]:
a_0 = sympy.symbols('a_0')
subs = {f: sympy.sqrt(a_0 * (x**2 + y**2))}
g8 = metric.Metric(twoform=g4.twoform.subs(subs).doit())
g8

In [ ]:
full_simplify(g8.matrix)

#### Curvature

In [ ]:
crs, rmn, rcs = curvature.compute_components(g8)

In [ ]:
curvature.display_components(crs)

In [ ]:
full_simplify(curvature.ricci_scalar(g8))

#### Parallel Transport

In [ ]:
full_simplify(geodesic.parallel_transport_equation(0, curve, param, g8))

In [ ]:
full_simplify(geodesic.parallel_transport_equation(1, curve, param, g8))

#### Geodesic

In [ ]:
a_0_val = 1.0
g8_num = metric.Metric(twoform=g8.twoform.subs({a_0: a_0_val}))

In [ ]:
ls = numpy.arange(0.0, 8.0, 0.001)
df8 = geodesic.numerical_sampler(g8_num, ls, (0.5, 0.5), tangent_scale=0.1)

In [ ]:
lim_df8 = boundary_filter(df8, x=(-1, 1), y=(-1, 1))

In [ ]:
if show_plots:
    fig = px.scatter(lim_df8, x="x", y='y', 
                     color='theta_0',
                     title="B8 Geodesic", 
                     height=600, width=600)

    fig.show()

## Applications

### C1 - Lower bound on size of universe

In [ ]:
M = Manifold('M', dim=3)
P = Patch('origin', M)

r, theta, phi = sympy.symbols('r theta phi', nonnegative=True)
a_0 = sympy.symbols('a_0', real=True, nonnegative=True)

cs = coords.CoordSystem('spherical', P, [r, theta, phi])
dr, dtheta, dphi = cs.base_oneforms()

ds2 = a_0 ** 2 * ((1 / (1 - r ** 2)) * tpow(dr, 2) + r ** 2 * (tpow(dtheta, 2) + 
                                                               sympy.sin(theta) ** 2 * tpow(dphi, 2)))

gc1 = metric.Metric(twoform=ds2)
gc1

In [ ]:
gc1.matrix

In [ ]:
R_0 = 1.7e-54 * (1 / units.meter) ** 2

In [ ]:
a0_val = numpy.sqrt(6 / R_0)

In [ ]:
V_bound = 2 * numpy.pi ** 2 * (a0_val) ** (3)

In [ ]:
V_bound

### C2 - Radial Geodesic

$$ L = \int_{\gamma} \sqrt{g_{\mu\nu}\frac{d x^{\mu}}{d\lambda}\frac{d x^{\nu}}{d\lambda}} d \lambda$$

Assume that radial geodesic implies that $$\frac{d}{d\lambda}\theta = \frac{d}{d\lambda}\phi = 0$$

$$ L_0 = \int_{\gamma} \sqrt{g_{rr}\left(\frac{d r}{d\lambda}\right)^2} d \lambda = \int_{0}^{r_0} \sqrt{g_{rr}}d r $$

In [ ]:
r_0 = sympy.symbols('r_0', real=True, nonnegative=True)

In [ ]:
soln = sympy.integrate(sympy.sqrt(gc1.matrix[0,0]), (r, 0, r_0))
soln

In [ ]:
soln.args[0][0]

$$ r_0 = \sin\left(\frac{L_0}{a_0}\right)$$

In [ ]:
L_0 = (46e9 * units.lightyear).to(units.meter)
L_0

In [ ]:
a0_val

In [ ]:
L_0 / a0_val

In [ ]:
r0_val = numpy.sin((L_0/a0_val).value)
r0_val

In [ ]:
v_obs = 4 / 3 * numpy.pi * r0_val ** 3
v_obs_scaled = v_obs * (a0_val) ** 3
v_obs_scaled

In [ ]:
v_obs_scaled / V_bound